# Imports

In [27]:
from ortools.sat.python import cp_model 
import numpy as np
import os
import wandb
import collections
import time
import pandas as pd

# Set up Weights & Biases

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cschmidl. Use `wandb login --relogin` to force relogin


True

# Custom Solutioncallback for Wandb

In [15]:
class WandbFeasibleSolutionsCallback(cp_model.CpSolverSolutionCallback):
    def __init__(self, jobs_data, assigned_task_type, all_tasks, all_machines, solution_array):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__jobs_data = jobs_data
        self.__assigned_task_type = assigned_task_type
        self.__all_tasks = all_tasks
        self.__all_machines = all_machines
        self.__solution_count = 0
        self.__start_time = time.time()

        # Contains solution dictionary where every key is the index of the machine pointing to an array of tuples
        # with (job_id, task_id, start_time, end_time (start + duration))
        #  Example: {0, [(7,2,84,93)]}
        self.__solution_array = solution_array
        wandb.log({'time': 0, 'solution_count': self.__solution_count})

    def _get_assigned_jobs(self):
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(self.__jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    self.__assigned_task_type(start = self.Value(
                        self.__all_tasks[job_id, task_id].start),
                                       job=job_id,
                                       index=task_id,
                                       duration=task[1]))
        return assigned_jobs

    def _add_solution(self, assigned_jobs, solution_id, makespan):
        # Create per machine solutions
        solution_type = None

        if self.Response().status == cp_model.OPTIMAL:
            solution_type = "Optimal"
        if self.Response().status == cp_model.FEASIBLE:
            solution_type = "Feasible"

        for i, machine in enumerate(self.__all_machines):
            #machine_tasks = []
            # Sort by starting time.
            assigned_jobs[machine].sort()
            machine_id = machine

            for j, assigned_task in enumerate(assigned_jobs[machine_id]):
                job_id = assigned_task.job
                task_id = assigned_task.index
                start = assigned_task.start
                duration = assigned_task.duration
                finish = start + duration

                self.__solution_array.append(
                    dict(
                        Machine=f"{machine_id}", 
                        Job=f"{job_id}",
                        Task=f"{task_id}", 
                        Start=start,
                        Duration=duration, 
                        Finish=finish,
                        Solution_id=solution_id,
                        Makespan=makespan, 
                        Solution_type=solution_type
                    )
                )

    def _print_per_machine_solution(self, assigned_jobs):
        # Create per machine output lines.
        output = ''
        for i, machine in enumerate(self.__all_machines):
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = 'Machine ' + str(machine) + ': '
            sol_line = '           '


            for j, assigned_task in enumerate(assigned_jobs[machine]):

                name = '(%i,%i)' % (assigned_task.job,
                                           assigned_task.index)
                # Add spaces to output to align columns.
                sol_line_tasks += '%-15s' % name
                start = assigned_task.start
                duration = assigned_task.duration
                sol_tmp = '[%i,%i]' % (start, start + duration)
                
                # Add spaces to output to align columns.
                sol_line += '%-15s' % sol_tmp

            sol_line += '\n'
            sol_line_tasks += '\n'
            output += sol_line_tasks
            output += sol_line

        # Finally print the solution found.
        print(f'Optimal Schedule Length: {self.ObjectiveValue()}')
        print(output)

    def on_solution_callback(self):
        """Called on each new solution."""
        current_time = time.time()
        obj = self.ObjectiveValue()
        self.__solution_count += 1
        wandb.log({'time': current_time - self.__start_time, 'make_span': obj, 'solution_count': self.__solution_count})

        assigned_jobs = self._get_assigned_jobs()
        self._add_solution(assigned_jobs, self.__solution_count, obj)

    def solution_count(self):
        """Returns the number of solutions found."""
        return self.__solution_count

# CPJobShopSolver-class

In [20]:
class CPJobShopSolver:
    def __init__(self, filename="data/taillard_instances/ta01.txt"):
        # Load the problem instance
        self.jobs_count, self.machines_count, self.jobs_data = self._load_instance(filename)

        self.filename = filename
        self.instance_name = os.path.split(filename)[-1].split(sep=".")[0].upper()

        self.model = None
        self.solver = None
        self.solver_parameters = None

        self.all_machines = None
        self.all_tasks = None
        self.assigned_task_type = None
        self.horizon = None

        self.solution_array = []

        self._init_cp_model()

    def _load_instance(self, filename):
        print(filename)
        jobs_data = []
        jobs_count = 0
        machines_count = 0

        # Refactor with logging behavior?
        if os.path.exists(filename) and os.path.isfile(filename):
            print(f"Loading instance from file: {filename}")

            with open(filename) as f:
                line_str = f.readline()
                line_count = 1

                while line_str:
                    data = []
                    split_data = line_str.split()
                    if line_count == 1:
                        jobs_count, machines_count = int(split_data[0]), int(split_data[1])
                    else:
                        i = 0
                        while i < len(split_data):
                            machine, time = int(split_data[i]), int(split_data[i+1])
                            data.append((machine, time))
                            i += 2
                        jobs_data.append(data)
                    line_str = f.readline()
                    line_count += 1

        else:
            print(f"File not found: {filename}.")
            return

        print(f"Successfully loaded instance {filename} with {jobs_count} jobs and {machines_count} machines.")
        return jobs_count, machines_count, np.array(jobs_data)

    def _init_cp_model(self):
        print(f"Initializing cp solver...")
        self.model = cp_model.CpModel()

        self.all_machines = range(self.machines_count)

        # Computes the horizon dynamically as the sum of all durations.
        self.horizon = sum(task[1] for job in self.jobs_data for task in job)
        print(f"Horizon is: {self.horizon}")

        # Named tuple to store information about created variables.
        task_type = collections.namedtuple('task_type', 'start end interval')
        # Named tuple to manipulate solution information.
        self.assigned_task_type = collections.namedtuple('assigned_task_type',
                                                    'start job index duration')

        # Creates job intervals and add to the corresponding machine lists.
        self.all_tasks = {}
        machine_to_intervals = collections.defaultdict(list)

        for job_id, job in enumerate(self.jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                duration = task[1]
                suffix = '_%i_%i' % (job_id, task_id)
                start_var = self.model.NewIntVar(0, self.horizon, 'start' + suffix)
                end_var = self.model.NewIntVar(0, self.horizon, 'end' + suffix)
                interval_var = self.model.NewIntervalVar(start_var, duration, end_var,
                                                    'interval' + suffix)
                self.all_tasks[job_id, task_id] = task_type(start=start_var,
                                                       end=end_var,
                                                       interval=interval_var)
                machine_to_intervals[machine].append(interval_var)

        # Create and add disjunctive constraints.
        for machine in self.all_machines:
            self.model.AddNoOverlap(machine_to_intervals[machine])

        # Precedences inside a job.
        for job_id, job in enumerate(self.jobs_data):
            for task_id in range(len(job) - 1):
                self.model.Add(self.all_tasks[job_id, task_id +
                                    1].start >= self.all_tasks[job_id, task_id].end)

        # Makespan objective.
        obj_var = self.model.NewIntVar(0, self.horizon, 'makespan')
        self.model.AddMaxEquality(obj_var, [
            self.all_tasks[job_id, len(job) - 1].end
            for job_id, job in enumerate(self.jobs_data)
        ])
        self.model.Minimize(obj_var)


    def _get_assigned_jobs(self, jobs_data, assigned_task_type, all_tasks, solver):
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(start = solver.Value(
                        all_tasks[job_id, task_id].start),
                                       job=job_id,
                                       index=task_id,
                                       duration=task[1]))
        return assigned_jobs

    def solve(self, max_time=10.0, use_wandb=True):
        # Initialize a new run

        if use_wandb:

            run = wandb.init(
                project="Example-project",
                notes="CP solver",
                group="constraint-programming",
                job_type=f"CP - {self.instance_name}",
                tags=["cp", "baseline", f"{self.instance_name}"]    
            )

            wandb.config.update({
                "instance_path": self.filename, 
                "max_time": max_time
            })

        self.solver = cp_model.CpSolver()

        if max_time != 0:
            self.solver.parameters.max_time_in_seconds = max_time

        if use_wandb:
            wandb_callback = WandbFeasibleSolutionsCallback(self.jobs_data, self.assigned_task_type, self.all_tasks, self.all_machines, self.solution_array)
            status = self.solver.Solve(self.model, wandb_callback)
        else:
            status = self.solver.Solve(self.model)
            

        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            status_as_string = "Optimal" if status == cp_model.OPTIMAL else "Feasible"

            assigned_jobs = self._get_assigned_jobs(self.jobs_data, self.assigned_task_type, self.all_tasks, self.solver)

            self._add_solution(assigned_jobs, wandb_callback.solution_count()+1, self.solver.ObjectiveValue(), status_as_string)

            # Upload solution artifact to wandb
            if use_wandb:
                df = pd.DataFrame(self.solution_array)
                wandb.log({f"{self.instance_name}_cp_solutions": wandb.Table(dataframe=df)})

        else:
            print('No solution found.')

        if use_wandb:
            run.finish()

        return self.solution_array

    def _add_solution(self, assigned_jobs, solution_id, makespan, solution_type):
        # Create per machine solutions

        for i, machine in enumerate(self.all_machines):
            assigned_jobs[machine].sort()
            machine_id = machine

            for j, assigned_task in enumerate(assigned_jobs[machine_id]):
                job_id = assigned_task.job
                task_id = assigned_task.index
                start = assigned_task.start
                duration = assigned_task.duration
                finish = start + duration

                self.solution_array.append(
                    dict(
                        Machine=f"{machine_id}", 
                        Job=f"{job_id}",
                        Task=f"{task_id}", 
                        Start=start,
                        Duration=duration, 
                        Finish=finish,
                        Solution_id=solution_id,
                        Makespan=makespan, 
                        Solution_type=solution_type
                    )
                )

# Execute experiment

In [ ]:
INSTANCE = "../../data/taillard_instances/ta01.txt"
cp_solver = CPJobShopSolver(filename=INSTANCE)
solution = cp_solver.solve()